# Funciones y tipos de datos recursivos

## Tipos de datos recursivos

### Tipo `List`

Una lista es una estructura de datos que representa una secuencia de valores del mismo tipo y de longitud finita. Podemos definirla como sigue:

Una lista puede ser:
- Una secuencia vacia de elementos
- Una secuencia integrada por un primer elemento y otra lista de elementos. Estos conceptos se denominan **cabeza** y **cola** respectivamente.

Así pues, el tipo lista puede ser definido mediante la siguiente equación:

Sea $ListaT$ una lista integrada por elementos de tipo $T$. Definimos $ListaT=1+T*ListaT$ tal que $1$ representa la unidad

Podemos así implementar en Scala la siguiente lista (`List[T]`)

In [3]:
object StdDefinition {
    sealed abstract class List[T]
    case class NonEmpty[T](head: T, tail: List[T]) extends List[T]
    case class Empty[T]() extends List[T]
}

defined object StdDefinition

Sin embargo, en Scala las lista vacias son definidas mediante un objeto (`Nil`) y no mediante una clase, de modo que la implementación se moriaría como sigue:

In [4]:
object ActualDefinition{
    sealed abstract class List[+T]
    case class ::[T](head: T, tail: List[T]) extends List[T]
    case object Nil extends List[Nothing]
}

defined object ActualDefinition

### Azucar sintáctico para listas

In [1]:
// Podemos escribirlas con una notación clásica
val l1: List[Int] = ::(1, ::(2, ::(3, Nil)))

// O podemos usar azucar sintáctico
val l2: List[Int] = 1 :: 2 :: 3 :: Nil
val l3: List[Int] = List(1,2,3)

l1: List[Int] = List(1, 2, 3)
l2: List[Int] = List(1, 2, 3)
l3: List[Int] = List(1, 2, 3)

Tambien podemos aplicar pattern matching sobre listas para así operar

In [2]:
// De forma clásica
l1 match {
    case Nil => 0
    case ::(head, tail) => 1
}

// O con azucar sintáctico
l1 match {
    case Nil => 0
    case head :: tail => 1
}

l1 match {
    case List() => 0
    case List(h1, h2, h3) => 1
}

res1_0: Int = 1
res1_1: Int = 1
res1_2: Int = 1

## Funciones recursivas sobre listas

Al hacer definido las listas como una estructura de datos recursiva podemos operar con ellas usando funciones recursivas.

Podemos ejemplificarlo con una función que calcule la longitud de una lista. Vamos a dar una versión iterativa y otra recursiva

In [6]:
//Implementacion iterativa
def lengthI[T](list: List[T]): Int = {
    var acc: Int = 0
    var aux: List[T] = list
    while (aux != Nil){
        // Esto no se debe hacer...
        aux = aux.asInstanceOf[::[T]].tail
        acc += 1
    }
    acc
}

// Implementación resursiva
def lengthR[T](list: List[T]): Int =
    list match {
        case Nil => 0
        case _ :: tail => 1 + lengthR(tail)
    }

defined function lengthI
defined function lengthR

In [7]:
// Resultados usando la versión iterativa
lengthI(List())
lengthI(List(1,2,3,4))

// Resultados usando la versión recursiva
lengthR(List())
lengthR(List(1,2,3,4))

res6_0: Int = 0
res6_1: Int = 4
res6_2: Int = 0
res6_3: Int = 4

Algunas observaciones:
- Las implementaciones recursivas se basan en un patrón de divide y vencerás por el que construimos la función basandonos en los diferentes tipos que vamos encontrando en cada caso. Este método facilita la implementación de una función en comparación con su versión iterativa
- Sin embargo la recursión es problemática para listas de muchos elementos ya que colapsan la pila del sistema

### Funciones de recursión de cola

Con el fin de solventar el problema del colapso de la pila del sistema, podemos usar una implementación basada en la recursión de cola. La idea principal es usar funciones auxiliares con el fin de no arrastrar las variables locales que tenemos siempre dentro de la función.

In [8]:
// Implementación de la longitus de una lista por recursión de cola
def lengthTR[A](l: List[A]) = {
    
    @annotation.tailrec
    def auxLength[A](count: Int, l: List[A]): Int =
        l match {
            case Nil => count
            case _ :: tail => auxLength(count + 1 , tail)
        }
    
    auxLength(0, l)
}

defined function lengthTR

In [10]:
//Probamos el método
lengthTR(List())
lengthTR(List(1,2,3))

res9_0: Int = 0
res9_1: Int = 3

Ahora podemos implementar un método que nos cree una lista constante de una longitud dada para probar la eficiencia de todos los métodos que hemos implementado

In [12]:
// Iterativamente
def constantListI[A](value: A, size: Int): List[A] = {
    var list: List[A] = Nil
    for (i <- 1 to size)
        list = value :: list
    return list
}

// Con recursión de cola
def constrantListTR[A](value: A, size: Int): List[A] = {
    
    def constantListTRAux(list: List[A], size: Int): List[A] = {
        if (size == 0) list
        else constantListTRAux(value :: list, size - 1)
    } 
    
    constantListTRAux(Nil, size)
}

defined function constantListI
defined function constrantListTR

Ahora podemos probar los métodos de longitud

In [13]:
// Iterativo
lengthI(constrantListTR(0, 1000000))

res12: Int = 1000000

In [14]:
// Con recursión de cola
lengthTR(constrantListTR(0, 1000000))

res13: Int = 1000000

In [19]:
// Con recursión simple (la pila desborda)
try
    lengthR(constrantListTR(0, 1000000))
catch {
    case _ => println("Stack overflow")
}

Stack overflow


res18: AnyVal = ()

## Testing unitario con `scalatest`

La libreria de `scalatest` nos permite implementar test unitarios para las funciones que programemos, de modo que usando el método `shouldBe` podemos comprobar los resultados reales con los esperados.

In [20]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

Podemos hacer un test con las funciones de longitud que declaramos anteriormente:

In [21]:
class TestLength(lengthF: List[Int] => Int) extends FlatSpec with Matchers{
    "length" should "work" in {
        lengthF(List()) shouldBe 0 
        lengthF(List(1)) shouldBe 1 
        lengthF(List(1,2,3,4)) shouldBe 4
    }
}

defined class TestLength

Mediante el método `run` ejecutamos el test

In [22]:
run(new TestLength(lengthR))

cmd20$Helper$TestLength:
length
- should work


## Algunos ejemplos de métodos sobre listas

### Sumar los números que contiene una lista

En primer lugar definimos un test para comprobar el comportamiento de nuestra función y a partir de ahí programarla (TDD).

In [23]:
class TestSum(sum: List[Int] => Int) extends FlatSpec with Matchers{
    "sum" should "work" in {
        sum(List()) shouldBe 0 
        sum(List(1)) shouldBe 1 
        sum(List(1,2,3,4)) shouldBe 10
    }
}

defined class TestSum

In [25]:
// Definición recursiva
def sumR(l: List[Int]): Int = {
    l match {
        case Nil => 0
        case head :: tail => head + sum(tail)
    }
}

defined function sumR

In [26]:
run(new TestSum(sumR))

cmd22$Helper$TestSum:
sum
- should work


In [27]:
// Definición con recursión de cola
def sumTR(l: List[Int]): Int = {
    
    def sumTRAux(count: Int, list: List[Int]): Int = {
        list match {
            case Nil => count
            case head :: tail => sumTRAux(count + head, tail)
        }
    }
    sumTRAux(0, l)
}

defined function sumTR

In [28]:
run(new TestSum(sumTR))

cmd22$Helper$TestSum:
sum
- should work


### Multiplicar los elementos que contiene una lista

In [29]:
class TestProduct(product: List[Int] => Int) extends FlatSpec with Matchers{
    "product" should "work" in {
        product(List()) shouldBe 1 
        product(List(1)) shouldBe 1 
        product(List(1,2,3,4)) shouldBe 24
        product(List(1,0,5,6,7)) shouldBe 0
    }
}

defined class TestProduct

In [30]:
// Definición recursiva
def multR(l: List[Int]): Int = {
    l match {
        case Nil => 1
        case head :: tail =>
            head * multR(tail)
    }
}

defined function multR

In [31]:
run(new TestProduct(multR))

cmd28$Helper$TestProduct:
product
- should work


In [32]:
// Optimización en caso de encontrar 0 en la lista, devolvemos directamente 0 como resultado
def multROpt(l: List[Int]): Int = {
    l match {
        case Nil => 1
        case 0 :: _ => 0
        case head :: tail => 
            head * multROpt(tail)
    }
}

defined function multROpt

In [33]:
run(new TestProduct(multROpt))

cmd28$Helper$TestProduct:
product
- should work


### Comprobar que un elemento pertenece a una lista

In [34]:
class TestMember(member: (List[Int], Int) => Boolean) extends FlatSpec with Matchers{
    "member" should "work" in {
        member(List(), 6) shouldBe false
        member(List(1), 1) shouldBe true
        member(List(1), 3) shouldBe false
        member(List(1,2,3,4), 4) shouldBe true
    }
}

defined class TestMember

In [38]:
// Implementación recursiva
def member[A](list: List[A], value: A): Boolean = {
    list match {
        case Nil => false
        case head :: tail => {
            if (head == value) true
            else member(tail, value)
        }
    }
}

defined function member

In [39]:
run(new TestMember(member[Int]))

cmd33$Helper$TestMember:
member
- should work


In [41]:
// implementación optimizada
def memberOpt[A](list: List[A], value: A): Boolean = {
    list match {
        case Nil => false
        case `value` :: _ => true
        case _ :: tail => member(tail, value)
    }
}

defined function memberOpt

In [42]:
run(new TestMember(memberOpt[Int]))

cmd33$Helper$TestMember:
member
- should work


### Obtener el último elemento de una lista

In [43]:
class TestLast(last: List[Int] => Option[Int]) extends FlatSpec with Matchers{
    "last" should "work" in {
        last(List()) shouldBe None
        last(List(1)) shouldBe Some(1)
        last(List(1,2,3,4)) shouldBe Some(4)
    }
}

defined class TestLast

In [44]:
def last[A](list: List[A]): Option[A] = {
    list match {
        case Nil => None
        case head :: Nil => Some(head)
        case _ :: tail => last(tail)
    }
}

defined function last

In [45]:
run(new TestLast(last[Int]))

cmd42$Helper$TestLast:
last
- should work


### Insertar un elemento al final de una lista

In [46]:
class TestInsertLast(insertLast: (List[Int], Int) => List[Int]) 
extends FlatSpec with Matchers{
    "insertLast" should "work" in {
        insertLast(List(), 6) shouldBe List(6)
        insertLast(List(1), 1) shouldBe List(1,1)
        insertLast(List(1,2,3,4), 4) shouldBe List(1,2,3,4,4)
    }
}

defined class TestInsertLast

In [47]:
 def insertLast[A](l: List[A], value: A): List[A] = 
    l match {
        case Nil => value :: Nil
        case head :: tail => head :: insertLast(tail, value)
    }

defined function insertLast

In [48]:
run(new TestInsertLast(insertLast[Int]))

cmd45$Helper$TestInsertLast:
insertLast
- should work


### Invertir una lista

In [49]:
class TestReverse(reverse: List[Int] => List[Int]) extends FlatSpec with Matchers{
    "reverse" should "work" in {
        reverse(List()) shouldBe List()
        reverse(List(1)) shouldBe List(1)
        reverse(List(1,2,3)) shouldBe List(3,2,1)
    }
}

defined class TestReverse

In [54]:
// Implementación usando insertLast (muy ineficiente)
def reverse[A](l: List[A]): List[A] = 
    l match {
        case Nil => Nil
        case head :: tail => insertLast(reverse(tail), head) // Podemos usar la notación: reverse(tail) :+ head
    }

defined function reverse

In [53]:
run(new TestReverse(reverse[Int]))

cmd48$Helper$TestReverse:
reverse
- should work


In [58]:
// Optimización usando recursión de cola
def reverseTR[A](list: List[A]): List[A] = {
    def reverseAux(acc: List[A], list: List[A]): List[A] = 
        list match {
            case Nil => acc
            case head :: tail => 
                reverseAux(head :: acc, tail)
        }
    
    reverseAux(Nil, list)
}

defined function reverseTR

In [59]:
run(new TestReverse(reverseTR[Int]))

cmd48$Helper$TestReverse:
reverse
- should work


### Concatenar dos listas

In [57]:
class TestConcatenate(concatenate: (List[Int], List[Int]) => List[Int]) 
extends FlatSpec with Matchers{
    "concatenate" should "work" in {
        concatenate(List(), List()) shouldBe List()
        concatenate(List(1), List()) shouldBe List(1)
        concatenate(List(), List(1)) shouldBe List(1)
        concatenate(List(1,2,3), List(1,3)) shouldBe List(1,2,3,1,3)
        concatenate(List(1,2,3), List(4,5,6)) shouldBe List(1,2,3,4,5,6)
    }
}

defined class TestConcatenate

In [61]:
// Implementación con recursión clásica
def concatenate[A](l1: List[A], l2: List[A]): List[A] = {
    l1 match {
        case Nil => l2
        case head :: tail => head :: concatenate(tail, l2)
    }
}

defined function concatenate

In [63]:
run(new TestConcatenate(concatenate[Int]))

cmd56$Helper$TestConcatenate:
concatenate
- should work


In [64]:
// Implementación con recursión de cola
def concatenateTR[A](list1: List[A], list2: List[A]): List[A] = {

    def concAux(acc: List[A], list: List[A]): List[A] = 
        list match {
            case Nil => acc
            case head :: tail => 
                concAux(head :: acc, tail)
        }
    
    concAux(list2, concAux(Nil, list1))
}

defined function concatenateTR

In [65]:
run(new TestConcatenate(concatenateTR[Int]))

cmd56$Helper$TestConcatenate:
concatenate
- should work


## Árboles binarios

Un arbol se define como una estructura de datos recursiva integrada por un elemento **raiz** y dos nodos hijo llamados **izquierdo** y **derecho**

Algebraicamente por tanto podemos definir un árbol binario como: $Arbol= 1 + (HijoIzquierdo * Raíz * HijoDerecho)$

Podemos definir el TAD árbol en Scala como sigue:

In [66]:
// type Tree[A] = 1 + Tree[A] * A * Tree[A]

sealed abstract class Tree[A]
case class Empty[A]() extends Tree[A]
case class Node[A](left: Tree[A], root: A, right: Tree[A]) extends Tree[A]

defined class Tree
defined class Empty
defined class Node

Sobre un arbol además podemos efectuar una serie de operaciones

In [68]:
object Tree{
    
    // Crea un arbol vacio
    def void[A]: Tree[A] = 
        Empty()
    
    // Crea un nodo hoja
    def leaf[A](a: A): Node[A] = 
        Node(Empty(), a, Empty())
    
    // Coloca un subárbol como hijo derecho y de raíz a
    def right[A](a: A, tree: Tree[A]): Node[A] = 
        Node(Empty(), a, tree)
    
    // Coloca un subárbol como hijo izquierdo y de raíz a
    def left[A](tree: Tree[A], a: A): Node[A] = 
        Node(tree, a, Empty())
    
    // Crea un nodo con raiz a y sus hijos como subárboles
    def node[A](left: Tree[A], a: A, right: Tree[A]): Node[A] = 
        Node(left, a, right)
}

import Tree._

defined object Tree
import Tree._